### Este código é a primeira versão completa e correta. Passou pela verificação e correção de um especialista.
### Também é incorporada aqui a primeira versão da quota de sexo e substituição de homens por mulheres
### Resta inconsistência apenas com relação a vagas otidas e não preenchidas (caso Tocantins 2022)

In [1]:
# CONSTANTES E CONFIGURAÇÕES
import pandas as pd
import numpy as np

# Constantes
CLAUSULA_AGREMIACAO = 0.8   # Cláusula de desempenho para participação do partido/federação nas vagas distribuídas pelas sobras
CLAUSULA_QP = 0.10          # Cláusula de desempenho individual para ocupar vaga pelo QP
CLAUSULA_MEDIAS = 0.20      # Cláusula de desempenho individual para ocupar vaga pelo Médias
ENCODING = "ISO-8859-1"     # Este é o padrão adotado pelos arquivos do TSE
SEP = ";"                   # Este é o separador de dados (colunas) adotado pelos arquivos do TSE

# Possíveis status que um candidato pode assumir após apuração (suplentes são considerados não eleitos)
STATUS_NAO_ELEITO = "nao_eleito"
STATUS_ELEITO_QP = "eleito_qp"
STATUS_ELEITO_SOBRA = "eleito_sobra"

# Carga dos arquivos de dados
ARQ_VOTOS_CANDIDATO = "votacao_candidato_munzona_2022_BRASIL.csv"   # Aqui é necessário mudar o nome do arquivo conforme a UF desejada
ARQ_VOTOS_PARTIDO = "votacao_partido_munzona_2022_BRASIL.csv"       # Aqui é necessário mudar o nome do arquivo conforme a UF desejada
ARQ_CANDIDATOS = "consulta_cand_2022_BRASIL.csv"                    # Aqui é necessário mudar o nome do arquivo conforme a UF desejada
ARQ_VAGAS = "consulta_vagas_2022_BRASIL.csv"                        # Aqui é necessário mudar o nome do arquivo conforme a UF desejada

# Colunas que serão preservadas na base de votação dos candidatos
COLS_VOT_CAND = [
    "SG_UF", "SG_UE", "NM_UE", "DS_CARGO", "SQ_CANDIDATO", "NM_CANDIDATO",
    "NM_URNA_CANDIDATO", "TP_AGREMIACAO", "SG_PARTIDO", "SG_FEDERACAO",
    "QT_VOTOS_NOMINAIS_VALIDOS"
]

# Colunas que serão preservadas na base de votação dos partidos
COLS_VOT_PART = [
    "SG_UF", "SG_UE", "NM_UE", "DS_CARGO", "TP_AGREMIACAO", "SG_PARTIDO",
    "SG_FEDERACAO", "QT_TOTAL_VOTOS_LEG_VALIDOS", "QT_VOTOS_NOMINAIS_VALIDOS"
]

# Colunas que serão preservadas na base de informações dos candidatos
COLS_CAND = [
    "SG_UF", "SG_UE", "NM_UE", "DS_CARGO", "SQ_CANDIDATO", "NM_CANDIDATO",
    "NM_URNA_CANDIDATO", "TP_AGREMIACAO", "SG_PARTIDO", "SG_FEDERACAO",
    "SG_UF_NASCIMENTO", "DT_NASCIMENTO", "NR_TITULO_ELEITORAL_CANDIDATO",
    "DS_GENERO", "DS_GRAU_INSTRUCAO", "DS_ESTADO_CIVIL", "DS_COR_RACA", "DS_OCUPACAO"
]

# Colunas que serão preservadas na base de vagas em disputa
COLS_VAGAS = [
    "SG_UF", "SG_UE", "NM_UE", "DS_CARGO", "QT_VAGA"
]

# FUNÇÃO AUXILIARES

In [2]:
# FUNÇÕES AUXILIARES DE CARGA E PRÉ-PROCESSAMENTO DE DADOS

def carregar_dados():
    """Carrega os dados iniciais dos arquivos CSV filtrando colunas e cargo 'Deputado Federal'."""
    votos_cand_zona_df = pd.read_csv(ARQ_VOTOS_CANDIDATO, sep=SEP, encoding=ENCODING, usecols=COLS_VOT_CAND)
    votos_part_zona_df = pd.read_csv(ARQ_VOTOS_PARTIDO, sep=SEP, encoding=ENCODING, usecols=COLS_VOT_PART)
    cand_df = pd.read_csv(ARQ_CANDIDATOS, sep=SEP, encoding=ENCODING, usecols=COLS_CAND)
    vagas_df = pd.read_csv(ARQ_VAGAS, sep=SEP, encoding=ENCODING, usecols=COLS_VAGAS)

    # Filtra apenas o cargo de Deputado Federal
    votos_cand_zona_df = votos_cand_zona_df[votos_cand_zona_df["DS_CARGO"] == "Deputado Estadual"]
    votos_part_zona_df = votos_part_zona_df[votos_part_zona_df["DS_CARGO"] == "Deputado Estadual"]
    cand_df = cand_df[cand_df["DS_CARGO"] == "DEPUTADO ESTADUAL"]
    vagas_df = vagas_df[vagas_df["DS_CARGO"] == "Deputado Estadual"]

    return votos_cand_zona_df, votos_part_zona_df, cand_df, vagas_df

def preprocessar_dados(votos_cand_zona_df, votos_part_zona_df, cand_df, vagas_df):
    """Executa merges, agregações e cálculos iniciais necessários."""
    # Agregação dos votos nominais dos candidatos (somando zonas)
    votos_cand_df = votos_cand_zona_df.groupby("SQ_CANDIDATO", as_index=False).agg({"QT_VOTOS_NOMINAIS_VALIDOS":"sum"}).astype(int)

    # Definir agremiação no cand_df e votos_part_zona_df
    cand_df["AGREMIACAO"] = cand_df.apply(define_agremiacao, axis=1)
    votos_part_zona_df["AGREMIACAO"] = votos_part_zona_df.apply(define_agremiacao, axis=1)

    # Renomeia coluna de votos nominais das agremiações na base votos_part_zona_df para evitar confusão com votos nominais dos candidatos
    votos_part_zona_df.rename(columns={"QT_VOTOS_NOMINAIS_VALIDOS": "QT_VOTOS_NOMINAIS_AGREMIACAO"}, inplace=True)

    # Agregação dos votos de legenda e nominais das agremiações
    votos_part_df = votos_part_zona_df.groupby(["SG_UE", "AGREMIACAO"], as_index=False).agg({
        "QT_TOTAL_VOTOS_LEG_VALIDOS":"sum",
        "QT_VOTOS_NOMINAIS_AGREMIACAO":"sum"
    })

    # Merge do cand_df com votos dos candidatos (leva a informação de votos nominais para a base de candidatos) e cria o dataframe consolidado
    consolidado_df = pd.merge(
        cand_df,
        votos_cand_df[["SQ_CANDIDATO", "QT_VOTOS_NOMINAIS_VALIDOS"]],
        on="SQ_CANDIDATO",
        how="left"
    )

    # Incorporar a informação das vagas em disputa em cada município no dataframe consolidado
    consolidado_df = pd.merge(consolidado_df, vagas_df[["SG_UE","QT_VAGA"]], on="SG_UE", how="left")

    # Incorporar votos partidários no dataframe consolidado
    consolidado_df = pd.merge(
        consolidado_df,
        votos_part_df[["SG_UE","AGREMIACAO","QT_TOTAL_VOTOS_LEG_VALIDOS","QT_VOTOS_NOMINAIS_AGREMIACAO"]],
        on=["SG_UE", "AGREMIACAO"],
        how="left"
    )

    # Cálculo de votos válidos do município
    votos_part_df["VOTOS_AGREMIACAO"] = votos_part_df["QT_TOTAL_VOTOS_LEG_VALIDOS"] + votos_part_df["QT_VOTOS_NOMINAIS_AGREMIACAO"]
    votos_municipio_df = votos_part_df.groupby(["SG_UE"], as_index=False).agg({"VOTOS_AGREMIACAO":"sum"})
    votos_municipio_df.rename(columns={"VOTOS_AGREMIACAO": "VOTOS_VALIDOS_MUN"}, inplace=True)

    # Merge vagas no votos_municipio_df
    votos_municipio_df = pd.merge(votos_municipio_df, vagas_df[["SG_UE","QT_VAGA"]], on="SG_UE", how="left")

    # Cálculo QE
    votos_municipio_df["QE"] = votos_municipio_df.apply(lambda x: calcula_QE(x["VOTOS_VALIDOS_MUN"], x["QT_VAGA"]), axis=1).astype(int)

    # Incorpora QE no votos_part_df
    votos_part_df = pd.merge(votos_part_df, votos_municipio_df[["SG_UE","QE"]], on="SG_UE", how="left")

    # Cálculo QP
    votos_part_df["QP"] = np.floor(votos_part_df["VOTOS_AGREMIACAO"] / votos_part_df["QE"]).fillna(0).astype(int)

    # Incorpora QE, QP no consolidado_df
    consolidado_df = pd.merge(
        consolidado_df,
        votos_part_df[["SG_UE","AGREMIACAO","VOTOS_AGREMIACAO","QE","QP"]],
        on=["SG_UE","AGREMIACAO"],
        how="left"
    )

    # Marcar candidatos conforme cláusulas
    consolidado_df["candidato_supera_10pc"] = consolidado_df["QT_VOTOS_NOMINAIS_VALIDOS"] >= (consolidado_df["QE"] * CLAUSULA_QP)
    consolidado_df["candidato_supera_20pc"] = consolidado_df["QT_VOTOS_NOMINAIS_VALIDOS"] >= (consolidado_df["QE"] * CLAUSULA_MEDIAS)
    consolidado_df["agremiacao_supera_80pc"] = consolidado_df["VOTOS_AGREMIACAO"] >= (consolidado_df["QE"] * CLAUSULA_AGREMIACAO)

    # Ajustar tipos
    colunas_float = consolidado_df.select_dtypes(include='float').columns
    consolidado_df[colunas_float] = consolidado_df[colunas_float].fillna(0).astype(int)

    # Inicia todos candidatos como não eleitos
    consolidado_df["SITUACAO_CANDIDATO"] = STATUS_NAO_ELEITO

    return consolidado_df, votos_part_df, votos_municipio_df, vagas_df

def define_agremiacao(row):
    if row['TP_AGREMIACAO'].strip().lower() == 'federação':
        return row['SG_FEDERACAO']
    else:
        return row['SG_PARTIDO']
    
def calcula_QE(votos_validos, vagas):
    """Calcula o Quociente Eleitoral (QE) com base na regra de arredondamento."""
    qe_base = votos_validos / vagas
    frac = qe_base - np.floor(qe_base)
    return int(np.floor(qe_base)) if frac <= 0.5 else int(np.ceil(qe_base))


In [3]:
# FUNÇÕES DE DISTRIBUIÇÃO DE VAGAS

def distribuir_vagas_por_QP(consolidado_df):
    """Distribui vagas por Quociente Partidário (QP)."""
    lista_eleitos = []
    grupos = consolidado_df.groupby(["SG_UE", "AGREMIACAO"], as_index=False)
    for (ue, agrem), grupo in grupos:
        qp_agrem = int(grupo["QP"].iloc[0])
        if qp_agrem > 0:
            eleitos = grupo[grupo["candidato_supera_10pc"]].sort_values(
                "QT_VOTOS_NOMINAIS_VALIDOS", ascending=False
            ).head(qp_agrem)
            lista_eleitos.append(eleitos[["SQ_CANDIDATO"]])

    if lista_eleitos:
        eleitos_qp_df = pd.concat(lista_eleitos, ignore_index=True)
        consolidado_df.loc[consolidado_df["SQ_CANDIDATO"].isin(eleitos_qp_df["SQ_CANDIDATO"]), "SITUACAO_CANDIDATO"] = STATUS_ELEITO_QP
    return consolidado_df

def selecionar_agremiacao_sobra(sobras_mun_df, exige_80, qe):
    """
    Seleciona as agremiações que participarão da distribuição das sobras.
    A média é calculada como: VOTOS_AGREMIACAO / (VAGAS_OBTIDAS + 1).
    Se exige_80 for True: filtra apenas agremiações com votos >= 80% do QE,
    Caso contrário, considera todas as agremiações.
    """
    if exige_80:
        df_filtro = sobras_mun_df[sobras_mun_df["VOTOS_AGREMIACAO"] >= (qe * CLAUSULA_AGREMIACAO)].copy()
    else:
        df_filtro = sobras_mun_df.copy()

    if df_filtro.empty:
        return None

    # Cálculo da média
    df_filtro["MEDIA"] = df_filtro["VOTOS_AGREMIACAO"] / (df_filtro["VAGAS_OBTIDAS"] + 1)
    df_filtro = df_filtro.sort_values("MEDIA", ascending=False)

    return df_filtro # no meu código antigo eu retornava apenas a agremiação com maior média, o que dava problema quando o candidato não tinha 20% do QE

def selecionar_candidato_sobra(cand_mun_df, agrem_escolhida, exige_20, qe):
    """
    Seleciona o candidato mais votado da agremiação escolhida que atenda às cláusulas.
    Se exige_20 for True: candidato deve ter votos >= 20% do QE.
    Caso não haja candidatos com 20%, relaxa a exigência (chamado com exige_20 = False).
    """
    if exige_20:
        candidatos = cand_mun_df[
            (cand_mun_df["AGREMIACAO"] == agrem_escolhida) &
            (cand_mun_df["SITUACAO_CANDIDATO"] == "nao_eleito") &
            (cand_mun_df["candidato_supera_20pc"] == True)
        ].copy()
    else:
        candidatos = cand_mun_df[
            (cand_mun_df["AGREMIACAO"] == agrem_escolhida) &
            (cand_mun_df["SITUACAO_CANDIDATO"] == "nao_eleito")
        ].copy()

    if candidatos.shape[0] > 0:
      candidatos = candidatos.sort_values("QT_VOTOS_NOMINAIS_VALIDOS", ascending=False)
      #display(candidatos)
      return candidatos.iloc[0]["SQ_CANDIDATO"]
    else:
      return None

def atualizar_candidato_sobra_e_log(consolidado_df, cand_mun_df, sobras_mun_df, controle_log, ue_id, vagas_sobra, exige_80, exige_20, agrem_escolhida, media, cand_escolhido):
    """
    Atualiza o status do candidato no consolidado_df e cand_mun_df, incrementa a vaga obtida pela agremiação
    e registra a operação no log.
    """
    if cand_escolhido is not None:
        # Atualiza status do candidato
        consolidado_df.loc[
            (consolidado_df["SG_UE"] == ue_id) & (consolidado_df["SQ_CANDIDATO"] == cand_escolhido),
            "SITUACAO_CANDIDATO"
        ] = "eleito_sobra"
        cand_mun_df.loc[cand_mun_df["SQ_CANDIDATO"] == cand_escolhido, "SITUACAO_CANDIDATO"] = "eleito_sobra"

        # Atualizar vagas obtidas pela agremiação
        sobras_mun_df.loc[sobras_mun_df["AGREMIACAO"] == agrem_escolhida, "VAGAS_OBTIDAS"] += 1

        # Registrar no log
        controle_log = registrar_log(controle_log, ue_id, vagas_sobra, exige_80, exige_20, agrem_escolhida, media, cand_escolhido, True)
    else:
        # Não foi encontrado candidato, ainda assim registrar a tentativa
        controle_log = registrar_log(controle_log, ue_id, vagas_sobra, exige_80, exige_20, agrem_escolhida, media, None, False)

    return consolidado_df, cand_mun_df, sobras_mun_df, controle_log

def distribuir_sobras(consolidado_df, votos_municipio_df, votos_part_df):
  print("Executa distribuir_sobras...")
  # Cálculo de vagas sobrantes
  vagas_qp_df = (consolidado_df[consolidado_df["SITUACAO_CANDIDATO"] == "eleito_qp"]
                  .groupby("SG_UE", as_index=False)
                  .size())
  vagas_qp_df.rename(columns={"size": "VAGAS_QP"}, inplace=True)

  vagas_sobra_df = pd.merge(
      votos_municipio_df[["SG_UE","QT_VAGA","QE"]],
      vagas_qp_df,
      on="SG_UE",
      how="left"
  )
  vagas_sobra_df["VAGAS_QP"] = vagas_sobra_df["VAGAS_QP"].fillna(0)
  vagas_sobra_df["VAGAS_SOBRA"] = vagas_sobra_df["QT_VAGA"] - vagas_sobra_df["VAGAS_QP"]

  # DataFrame de controle (log)
  controle_log = pd.DataFrame(columns=[
      "UE", "Vagas Sobra", "Exige 80%", "Exige 20%", "Agremiação Escolhida",
      "Média", "Candidato Escolhido", "Status Atualizado"
  ])

  # Loop por município
  for _, row_mun in vagas_sobra_df.iterrows():
      ue_id = row_mun["SG_UE"]
      vagas_sobra = int(row_mun["VAGAS_SOBRA"])
      qe = row_mun["QE"]

      if vagas_sobra <= 0:
          continue

      sobras_mun_df = votos_part_df[votos_part_df["SG_UE"] == ue_id].copy()
      sobras_mun_df["VAGAS_OBTIDAS"] = sobras_mun_df["QP"]
      cand_mun_df = consolidado_df[consolidado_df["SG_UE"] == ue_id].copy()

      # Define exigências iniciais
      # Primeira tentativa: exige 80% e 20%
      exige_80 = (sobras_mun_df["VOTOS_AGREMIACAO"] >= (qe * CLAUSULA_AGREMIACAO)).any()
      exige_20 = exige_80  # se tiver agremiação com 80%, exige também 20% para candidatos

      while vagas_sobra > 0 :
        # Seleciona as agremiações que superam a cláusula de desempenho mínimo
        df_agrem_aptas  = selecionar_agremiacao_sobra(sobras_mun_df, exige_80, qe)

        # Se não encontrou agremiação, relaxa a cláusula dos 80%
        if df_agrem_aptas is None:
          if exige_80:
            exige_80 = False
          else:
            break # Se nem relaxado a cláusula não há agremiações, não há mais o que fazer
        else:
          qtd_agrem = df_agrem_aptas.shape[0] # quantidade de agremiações que superam a cláusula de desempenho

          # Loop no dataframe das agremiações e escolha do candidato
          for i, row_agrem in enumerate(df_agrem_aptas.itertuples()):
            agrem_escolhida, media = row_agrem.AGREMIACAO, row_agrem.MEDIA

            cand_escolhido = selecionar_candidato_sobra(cand_mun_df, agrem_escolhida, exige_20, qe)

            if cand_escolhido is None:
              # Não há candidato com 20% e verificou em todas agremiações
              # se exige_20 estiver True, vamos relaxar a exigência de 20%
              if exige_20 and i+1 == qtd_agrem:
                  exige_20 = False
                  # Tenta novamente sem a exigência de 20% (no mesmo loop)
                  continue

              elif not exige_20:
                  # Mesmo sem a exigência de 20% não achou candidato, registra e reduz vagas_sobra
                  consolidado_df, cand_mun_df, sobras_mun_df, controle_log = atualizar_candidato_sobra_e_log(
                      consolidado_df, cand_mun_df, sobras_mun_df, controle_log,
                      ue_id, vagas_sobra, exige_80, exige_20, agrem_escolhida, media, None
                  )
                  vagas_sobra -= 1
                  break
              else:
                continue
            else:
                # Candidato encontrado
                consolidado_df, cand_mun_df, sobras_mun_df, controle_log = atualizar_candidato_sobra_e_log(
                    consolidado_df, cand_mun_df, sobras_mun_df, controle_log,
                    ue_id, vagas_sobra, exige_80, exige_20, agrem_escolhida, media, cand_escolhido
                )
                vagas_sobra -= 1
                break
  return consolidado_df, controle_log

def registrar_log(controle_log, ue_id, vagas_sobra, exige_80, exige_20, agrem, media, candidato_escolhido, status_atualizado):
    """
    Registra informações sobre a distribuição da sobra no log.
    """
    nova_linha = {
        "UE": ue_id,
        "Vagas Sobra": vagas_sobra,
        "Exige 80%": exige_80,
        "Exige 20%": exige_20,
        "Agremiação Escolhida": agrem,
        "Média": media,
        "Candidato Escolhido": candidato_escolhido,
        "Status Atualizado": status_atualizado
    }
    return pd.concat([controle_log, pd.DataFrame([nova_linha])], ignore_index=True)

# MAIN

In [4]:
# FLUXO PRINCIPAL - APURAÇÃO COM CLÁUSULAS DE DESEMPENHO

if __name__ == "__main__":

    votos_cand_zona_df, votos_part_zona_df, cand_df, vagas_df = carregar_dados()

    # Pré-processar e calcular parâmetros básicos (QE, QP, cláusulas)
    consolidado_df, votos_part_df, votos_municipio_df, vagas_df = preprocessar_dados(
        votos_cand_zona_df, votos_part_zona_df, cand_df, vagas_df
    )

    # Distribuir vagas pelo Quociente Partidário (QP)
    consolidado_df = distribuir_vagas_por_QP(consolidado_df)

    # Distribuir sobras
    consolidado_df, controle_log = distribuir_sobras(consolidado_df, votos_municipio_df, votos_part_df)

Executa distribuir_sobras...


C:\Users\p_8201\AppData\Local\Temp\ipykernel_7936\2861003020.py:192: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([controle_log, pd.DataFrame([nova_linha])], ignore_index=True)


In [5]:
# EXPORTAÇÃO DOS RESULTADOS PRINCIPAIS PARA ARQUIVOS CSV
consolidado_df.to_csv("resultado_final.csv", sep=SEP, index=False, encoding=ENCODING) # Arquivo com todos os dados e situação dos candidatos
controle_log.to_csv("controle.csv", sep=SEP, index=False, decimal = ",", encoding=ENCODING) # Arquivo de controle da distribuição das sobras

# GENERO

In [11]:
# VERIFICAR SE A QUOTA DE GÊNERO SERIA ATENDIDA

def arredonda_quotas(vagas, percentual):
    """
    Aplica a regra de arredondamento para o mínimo de vagas de mulheres (quota mínima).
    Exemplo: percentual = 0.20 (20%)
    ATENÇÃO: arredonda para baixo, se a fração for menor ou igual a 0.5, e para cima, se maior do que 05.
    """
    valor_base = vagas * percentual
    frac = valor_base - np.floor(valor_base)
    if frac <= 0.5:
        return int(np.floor(valor_base))
    else:
        return int(np.ceil(valor_base))

def verificar_quotas_genero(consolidado_df):
    """
    Verifica se as quotas mínimas de 20% de mulheres foram atingidas por município.
    Considera como eleitas: 'eleito_qp', 'eleito_sobra' e 'eleita_quota'.
    Retorna um DataFrame com a verificação:
    - MULHERES_ELEITAS: Número de mulheres efetivamente eleitas.
    - MIN_MULHERES: Mínimo exigido de mulheres (após arredondamento).
    - QUOTA_ATINGIDA: Indica se a quota mínima de mulheres foi alcançada.
    """
    # Consideramos como eleitas também aquelas com situação "eleita_quota"
    eleitos_df = consolidado_df[consolidado_df["SITUACAO_CANDIDATO"].isin(["eleito_qp","eleito_sobra","eleita_quota"])].copy()

    # Agrupar por município e contar quantas mulheres foram eleitas
    quotas_df = eleitos_df.groupby(["SG_UE","QT_VAGA"], as_index=False).agg(
        MULHERES_ELEITAS=("DS_GENERO", lambda x: (x == "FEMININO").sum()),
        TOTAL_ELEITOS=("SITUACAO_CANDIDATO", "count")
    )

    # Calcula o mínimo de mulheres exigido
    quotas_df["MIN_MULHERES"] = quotas_df["QT_VAGA"].apply(lambda x: arredonda_quotas(x, 0.2))

    # Verifica cumprimento da quota
    quotas_df["QUOTA_ATINGIDA"] = quotas_df["MULHERES_ELEITAS"] >= quotas_df["MIN_MULHERES"]

    return quotas_df

quotas_resultados_df = verificar_quotas_genero(consolidado_df)
quotas_resultados_df.to_csv("verificacao_quotas_genero.csv", sep=SEP, index=False, decimal = ",", encoding=ENCODING)

In [12]:
# Define o processo/função para a substituição de homens por mulheres
def substituir_homens_por_mulheres(consolidado_df, controle_log, quotas_resultados_df):
    """
    Tenta assegurar a quota mínima de 20% de mulheres por município, substituindo homens eleitos por sobra por mulheres não eleitas.
    - Não altera consolidado_df diretamente, cria uma cópia consolidado_quotas_df.
    - Usa verificar_quotas_genero(consolidado_quotas_df) para verificar o cumprimento da quota após cada substituição.
    - Quanto maior o número em 'Vagas Sobra', mais cedo a vaga foi atribuída. Portanto, ordenamos em ordem crescente para começar pelos homens que receberam as últimas vagas (com números menores).
    - O homem substituído passa a ter situação 'homem_substituido'.
    - A mulher que assume a vaga passa de 'nao_eleito' a 'eleita_quota'.
    """

    # Cria uma cópia para não alterar o consolidado_df original
    consolidado_quotas_df = consolidado_df.copy()

    # Verificar quotas iniciais
    quotas_resultados_df = verificar_quotas_genero(consolidado_quotas_df)
    municipios_nao_conforme = quotas_resultados_df[quotas_resultados_df["QUOTA_ATINGIDA"] == False]

    for _, row in municipios_nao_conforme.iterrows():
        ue_id = row["SG_UE"]
        min_mulheres = row["MIN_MULHERES"]

        # Verificar quotas para o município atual
        quotas_ue = quotas_resultados_df[quotas_resultados_df["SG_UE"] == ue_id]
        if quotas_ue.empty:
            continue

        mulheres_eleitas = int(quotas_ue["MULHERES_ELEITAS"].iloc[0])

        # Se já atende a quota, não faz nada
        if mulheres_eleitas >= min_mulheres:
            continue

        # Ordenar controle_log em ordem crescente de "Vagas Sobra"
        log_mun = controle_log[
            (controle_log["UE"] == ue_id) &
            (controle_log["Status Atualizado"] == True) &
            (controle_log["Candidato Escolhido"].notnull())
        ].copy()

        log_mun = log_mun.sort_values("Vagas Sobra", ascending=True)

        # Iterar pelas vagas de sobra em ordem crescente
        for _, sobra_row in log_mun.iterrows():
            # Recalcular quotas após cada tentativa
            quotas_resultados_df = verificar_quotas_genero(consolidado_quotas_df)
            quotas_ue = quotas_resultados_df[quotas_resultados_df["SG_UE"] == ue_id]
            if quotas_ue.empty:
                break

            mulheres_eleitas = int(quotas_ue["MULHERES_ELEITAS"].iloc[0])
            if mulheres_eleitas >= min_mulheres:
                # Quota atingida, interrompe o loop
                break

            cand_escolhido = sobra_row["Candidato Escolhido"]
            agrem_escolhida = sobra_row["Agremiação Escolhida"]

            # Informações do candidato atualmente eleito pela sobra
            info_candidato = consolidado_quotas_df[consolidado_quotas_df["SQ_CANDIDATO"] == cand_escolhido].iloc[0]
            genero_cand = info_candidato["DS_GENERO"]
            situacao_cand = info_candidato["SITUACAO_CANDIDATO"]

            # Somente substituímos se for homem e eleito_sobra
            if genero_cand != "FEMININO" and situacao_cand == "eleito_sobra":
                # Buscar a mulher mais bem votada, não eleita, da mesma agremiação e município
                mulheres_nao_eleitas = consolidado_quotas_df[
                    (consolidado_quotas_df["SG_UE"] == ue_id) &
                    (consolidado_quotas_df["AGREMIACAO"] == agrem_escolhida) &
                    (consolidado_quotas_df["DS_GENERO"] == "FEMININO") &
                    (consolidado_quotas_df["SITUACAO_CANDIDATO"] == "nao_eleito")
                ].copy()

                mulheres_nao_eleitas = mulheres_nao_eleitas.sort_values("QT_VOTOS_NOMINAIS_VALIDOS", ascending=False)

                if not mulheres_nao_eleitas.empty:
                    # Candidata substituta
                    mulher_substituta = mulheres_nao_eleitas.iloc[0]["SQ_CANDIDATO"]

                    # Atualizar consolidado_quotas_df:
                    consolidado_quotas_df.loc[
                        consolidado_quotas_df["SQ_CANDIDATO"] == cand_escolhido,
                        "SITUACAO_CANDIDATO"
                    ] = "homem_substituido"

                    consolidado_quotas_df.loc[
                        consolidado_quotas_df["SQ_CANDIDATO"] == mulher_substituta,
                        "SITUACAO_CANDIDATO"
                    ] = "eleita_quota"

                    # Recalcular quotas após substituição
                    quotas_resultados_df = verificar_quotas_genero(consolidado_quotas_df)
                    quotas_ue = quotas_resultados_df[quotas_resultados_df["SG_UE"] == ue_id]
                    if not quotas_ue.empty:
                        mulheres_eleitas = int(quotas_ue["MULHERES_ELEITAS"].iloc[0])
                        if mulheres_eleitas >= min_mulheres:
                            # Quota atingida, parar neste município
                            break

    return consolidado_quotas_df, quotas_resultados_df

In [13]:
# SUBSTITUIÇÃO DE HOMENS POR MULHERES
consolidado_quotas_df, quotas_resultados_df = substituir_homens_por_mulheres(consolidado_df, controle_log, quotas_resultados_df)
verificacao_quotas_genero_depois = verificar_quotas_genero(consolidado_quotas_df)


In [14]:
# EXPORTAÇÃO DOS RESULTADOS COM QUOTA DE MULHERES E SUBSTITUIÇÕES PARA ARQUIVOS CSV
consolidado_quotas_df.to_csv("resultado_final_quotas.csv", sep=SEP, index=False, decimal = ",", encoding=ENCODING)
verificacao_quotas_genero_depois.to_csv("verificacao_quotas_genero_depois.csv", sep=SEP, index=False, decimal = ",", encoding=ENCODING)